In [3]:
import pandas as pd
import numpy as np

# load_raw = pd.read_csv('load.csv',index_col=0)
# print(load_raw['load'+'1'].mean())
# with open("optimal_cloudlet.txt", "r") as f:  # 打开文件
#     optimal = int(f.read())  # 读取文件

# cloudlet数量
# cloudlet_number = load_raw.shape[1]
cloudlet_number = 10
# 任务数量
task_number = 100

# 任务大小 服从 均值=4600M条指令 的 均匀分布
task_size = np.random.uniform(500,8700,task_number).round()
# 移动设备处理能力(MIPS)
speed_moblieDevice = 600
# cloudlet处理能力(MIPS)
speed_cloudlet = 3846
# clooud处理能力(MIPS)
speed_cloud = 31790


# 任务所需传输数据量 服从 均值=750KB 的 均匀分布 假设下行的时间为上行的 0.6 倍
task_upload = np.random.uniform(200,1300,task_number).round()
# 边缘传输速率：带宽设为100mbps 理论传输率最大 12.8MB/s  实际传输吞吐量 4-11MB/S
trans_rate_clt = np.random.uniform(4,11,task_number).round(2)
# 云传输速率：带宽设为1Gbps 理论传输率最大 128MB/s  实际传输吞吐量 40-110MB/S
trans_rate_c = np.random.uniform(40,110,task_number).round(2)

# # 终端设备离cloudlet的距离 为(5,50)米
# distance_cloudlet = np.random.randint(5,50,cloudlet_number)
# nearest = np.argmin(distance_cloudlet)

# # 在cloudlet中的排队时间
# Tclt_queue = np.random.uniform(0,1.5,cloudlet_number)

# 功率
# 设 移动设备空闲时的功率为 5mW/sec 通信时的能耗为 20mW/sec   1w = 1 J/S     1mw = 1e-3 J/S
power_idle = 5
power_communication = 20
power_excute = 60
# 总的时间 和 能耗
Tm = Tclt = Tc = 0
Em = Eclt = Ec = 0

# 失败任务的个数
failureCount = 0


In [ ]:

for i in range(task_number):
    # 初始化某个任务的完成时间  temp0 表示在本地处理   temp1表示在最近cloudlet处理
    temp0 = temp1 = 0

    # 在本地执行任务花费的时间
    temp0 = task_size[i] / speed_moblieDevice
    
    # 终端设备到各个cloudlet的距离
    distance_cloudlet = np.random.randint(5,50,cloudlet_number)
    # 最近的cloudlet
    nearest = np.argmin(distance_cloudlet)

    # 上传到最近cloudlet的通信时延
    temp1_upload = (task_upload[i] / 1024) / trans_rate_clt[nearest]
    # 传播时延
    temp1_propagation = distance_cloudlet[nearest] / 3e8
    
    # 在cloudlet中的排队时间
    Tclt_queue = np.random.uniform(0,1.5,cloudlet_number)
    temp1_queue = Tclt_queue[nearest]

    # cloudlet处理时间
    temp1_excute = task_size[i] / speed_cloudlet

    temp1 = temp1_upload*1.6 + temp1_propagation + temp1_queue + temp1_excute

    # 最近cloudlet负载
    load_nearest = abs(np.random.normal(12, 6, 10))
    # 最佳cloudlet
    # optimal = np.random.randint(0,20)
    # 最佳cloudlet负载
    load_optimal = abs(np.random.normal(12, 6, 10))

    if temp0 <= temp1:
        # print('在本地执行')
        if temp0 >= 5:
            failureCount += 1
        Tm = Tm + temp0
        Em = Em + power_excute * temp0
    elif load_nearest.mean() <= 16 * 0.8 :
        # print('卸载到最近cloudlet')
        if temp1 >= 5:
            failureCount += 1
        Tclt = Tclt + temp1
        Eclt = Eclt +  power_communication * temp1_upload * 1.6 + power_idle * (temp1_propagation + temp1_queue + temp1_excute)
    elif load_optimal.mean() <= 16 * 0.8:
        # print('卸载到最佳cloudlet')
        # 上传到最近
        Topt_upload = temp1_upload
        # 传到最近 再到最佳
        Topt_propagation = (np.random.randint(5,50) / 3e8) * 2
        Topt_queue = np.random.uniform(0,0.7)
        Topt_excute = task_size[i] / speed_cloudlet
        if Topt_upload * 1.6 + Topt_propagation + Topt_queue + Topt_excute >= 5:
            failureCount += 1
        Tclt = Tclt + Topt_upload * 1.6 + Topt_propagation + Topt_queue + Topt_excute
        Eclt = Eclt +  power_communication * Topt_upload * 1.6 + power_idle * (Topt_propagation + Topt_queue + Topt_excute)
    else:
        # print('卸载到云')
        Tc_upload = (task_upload[i] / 1024) / trans_rate_c[i]
        Tc_propagation = np.random.randint(10000,500000) / 3e8
        Tc_excute = task_size[i] / speed_cloud
        if  Tc_upload * 1.6 + Tc_propagation  + Tc_excute>= 5:
            failureCount += 1
        Tc = Tc +  Tc_upload * 1.6 + Tc_propagation  + Tc_excute
        Ec = Ec + power_communication * Tc_upload * 1.6 + power_idle * (Tc_propagation + Tc_excute)
    
print("执行这些任务的时间约为：",round(Tm + Tclt + Tc,2))
print("执行这些任务的能耗约为：", round(Em + Eclt + Ec,2))
print("任务失败个数为：",failureCount)